# Введение в обработку естественного языка
## Урок 4. Тематическое моделирование. EM-алгоритм

In [52]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'
!pip install annoy
!pip install pymorphy2
!pip install stop_words
!pip install pyLDAvis

--2022-06-07 16:37:16--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-07 16:37:16--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc06ff8f177b02e1c7098a9b6e56.dl.dropboxusercontent.com/cd/0/inline/BmyXfTHAnlt-3rvuVc4hQ8_BcaDmF46tE7TyHbsmp1gsOByg1FUToC-XARpLVAflQY9PPY2zsIzhWixuz6aC0MiNLW8KHylvLghg-jb6jfCe2q-I9asTCP-PflZlMS5Wc1Xca756b3-FR7KSeHDZzQ850hN0__87UiOWObh9jEAklw/file# [following]
--2022-06-07 16:37:16--  https://uc06ff8f177b02e1c7098a9b6e56.dl.dropboxusercontent.com/cd/0/inline/BmyXfTHAnlt-3rvuVc4hQ8_BcaDmF46tE7TyHbsmp1gsOByg1FUToC-XARpLVAflQY9P

In [56]:
import os
import urllib.request
from scipy import spatial
import os
import urllib.request
from scipy import spatial
import pandas as pd
import numpy as np
import codecs
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import re
from html.parser import HTMLParser
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from string import punctuation
from pymystem3 import Mystem
from gensim.models import Word2Vec, FastText
import annoy
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### объединить в одну выборку

In [6]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative])
df = df.reset_index().drop('index', axis=1)

### провести исследование и выявить тематики о которых говорят в твитах

In [37]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
sw.update(pd.read_csv('/content/Russian.csv', header=None)[0].tolist())
exclude = set(punctuation)

def preprocess(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [39]:
df_pro = df['text'].apply(preprocess)

In [42]:
from gensim.models import *
from gensim import corpora

dictionary = corpora.Dictionary(df_pro)
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')

In [43]:
corpus = [dictionary.doc2bow(text) for text in df_pro]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [49]:
%time
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [50]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(0,
  '0.032*"rt" + 0.026*"любить" + 0.015*"вроде" + 0.014*"минута" + 0.013*"нужный" + 0.013*"2" + 0.012*"холодно" + 0.012*"боль" + 0.012*"эх" + 0.012*"умереть"'),
 (1,
  '0.027*"говорить" + 0.026*"rt" + 0.024*"последний" + 0.023*"друг" + 0.015*"остаться" + 0.014*"читать" + 0.013*"самый" + 0.013*"закончиться" + 0.013*"4" + 0.012*"обидно"'),
 (2,
  '0.024*"писать" + 0.024*"жаль" + 0.022*"rt" + 0.016*"понимать" + 0.015*"нравиться" + 0.013*"снег" + 0.013*"чтоть" + 0.013*"скучно" + 0.012*"улица" + 0.012*"вк"'),
 (3,
  '0.065*"rt" + 0.022*"плохо" + 0.016*"оо" + 0.013*"ужасно" + 0.013*"пиздец" + 0.013*"бояться" + 0.012*"смочь" + 0.010*"бывать" + 0.010*"хватать" + 0.009*"начинать"'),
 (4,
  '0.033*"делать" + 0.028*"скучать" + 0.025*"rt" + 0.021*"сидеть" + 0.020*"дом" + 0.020*"мама" + 0.018*"знать" + 0.017*"прийти" + 0.012*"дело" + 0.011*"слово"'),
 (5,
  '0.056*"завтра" + 0.036*"спать" + 0.035*"школа" + 0.035*"идти" + 0.027*"пойти" + 0.018*"rt" + 0.018*"ночь" + 0.017*"урок" + 0.015*"экзамен"

### сделать визуализацию кластеров тематик

In [57]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


### проинтерпритировать получившиеся тематики

- людей интересуют житейские/обыденные вопросы;
- нужно внимательней следить за непригодными для учета словами;
- подобный анализ должен быть лишь одной из ступеней NLP, так как даже одно слово, вырванное из контекста, может сломать судьбу.